Given an array of numbers, return whether any two sums to K.

For example, given [10, 15, 3, 7] and K of 17, return true since 10 + 7 is 17.

In [163]:
#naive approach
from itertools import combinations
def pair_sum_check(A, k):
    return any(i+j == k for i, j in combinations(A, 2))

#O(nlgn) approach
def binary_search(seq, t):
    min = 0
    max = len(seq) - 1
    while True:
        if max < min:
            return -1
        m = (min + max) // 2
        if seq[m] < t:
            min = m + 1
        elif seq[m] > t:
            max = m - 1
        else:
            return m

def pair_sum_check_2(A, k):
    return any(binary_search(A[i:], k-e) != -1 for i, e in enumerate(sorted(A)))
    
#O(n) approach - requires bounded range of numbers in A
from collections import defaultdict
def pair_sum_check_3(A, k):
    win_path = defaultdict(bool)
    for i in A:
        if win_path[k-i]:
            return True
        else:
            win_path[i] = True
    return False

#the probably faster in python approach
def pair_sum_check_4(A, k):
    win_path = set(A)
    return any(k-i in win_path for i in A)

#generalized version
def n_sum_check(A, k):
    win_path = set()
    for i in A:
        if k-i in win_path:
            return True
        else:
            new_paths = set()
            new_paths.add(i)
            for j in win_path:
                new_paths.add(i+j)
            win_path = win_path.union(new_paths)
    return False
        
    

Given an array of integers, return a new array such that each element at index i of the new array is the product of all the numbers in the original array except the one at i. Solve it without using division and in O(n) time.

For example, if our input was [1, 2, 3, 4, 5], the expected output would be [120, 60, 40, 30, 24]. If our input was [3, 2, 1], the expected output would be [2, 3, 6].

In [164]:
def product_complement(A):
    A2 = [1] * len(A)
    for i, _ in enumerate(A[1:], 1):
        A2[i] *= A[i-1] * A2[i-1]
    
    A3 = [1] * len(A)
    for i, _ in reversed(list(enumerate(A[:-1], 0))):
        A3[i] *= A[i+1] * A3[i+1] 
        
    return [i*j for i, j in zip(A2, A3)]

In [165]:

pair_sum_check_4([10, 15, 3, 7],11 )

False

In [149]:
import random
A = [random.choice(range(10000)) for _ in range(1000)]

%timeit pair_sum_check(A, random.randint(1,20000))
%timeit pair_sum_check_2(A, random.randint(1,20000))
%timeit pair_sum_check_3(A, random.randint(1,20000))
%timeit pair_sum_check_4(A, random.randint(1,20000))


7.72 ms ± 1.44 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
3.88 ms ± 146 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
99.3 µs ± 15 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
43 µs ± 679 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [167]:
%timeit n_sum_check(A, random.randint(1,20000))

The slowest run took 35.36 times longer than the fastest. This could mean that an intermediate result is being cached.
3.29 s ± 4.14 s per loop (mean ± std. dev. of 7 runs, 10 loops each)


This problem was asked by Google.

Given the root to a binary tree, implement serialize(root), which serializes the tree into a string, and deserialize(s), which deserializes the string back into the tree.

In [102]:
from itertools import count

class Node:
    
    def __init__(self, name):
        self.name = name
        self.left = None
        self.right = None


from queue import Queue



def serialize(root, delimiter=','):
    def serializer(root):
        q = Queue()
        q.put(root)
        while True: 
            if q.empty(): break
            root = q.get()
            if root.left: q.put(root.left)
            if root.right: q.put(root.right)
            yield root.name
     
    return delimiter.join([i for i in serializer(root)])

def deserialize(s, delimiter=','):
    nodes = [Node(name) for name in s.split(delimiter)]
    for index, node in enumerate(nodes[:len(nodes)//2]):
        left_index = index*2+1
        if left_index < len(nodes):
            node.left = nodes[left_index]
        right_index = index*2+2
        if right_index < len(nodes):
            node.right = nodes[right_index]
            
    return nodes[0]

    

In [110]:
import random
def test_serialize():
    labels = (str(random.randint(0,1000)) for _ in range(random.randint(1,1000)))
    serial = ','.join(labels)
    return serial == serialize(deserialize(serial))

This problem was asked by Stripe.

Given an array of integers, find the first missing positive integer in linear time and constant space. In other words, find the lowest positive integer that does not exist in the array. The array can contain duplicates and negative numbers as well.

For example, the input [3, 4, -1, 1] should give 2. The input [1, 2, 0] should give 3.

You can modify the input array in-place.

In [76]:
def find_first_missing_positive(A):
    bitmap = [False] * (max(A) + 1)
    for i in A:
        if i>0:
            bitmap[i-1] = True
    return bitmap.index(False) + 1

def find_first_missing_positive_in_place(A):
    for i,e in enumerate(A):
        A[i] = max(0,e)

    A.append(0)
    for e in A:
        if e < len(A)+1:

            A[abs(e)-1] = min(A[e], -A[e])
    A[-1] = 1

    for i, e in enumerate(A):
        if e > 0: return i+1


                                         

This problem was asked by Jane Street.

cons(a, b) constructs a pair, and car(pair) and cdr(pair) returns the first and last element of that pair. For example, car(cons(3, 4)) returns 3, and cdr(cons(3, 4)) returns 4.

Given this implementation of cons:

def cons(a, b):
    return lambda f : f(a, b)
Implement car and cdr.


In [12]:
def cons(a,b):
    return lambda f: f(a,b)

def car(pair):
    return pair(lambda a,b: a)

def cdr(pair):
    return pair(lambda a,b: b)